In [ ]:
using FFTW
using Plots
using LaTeXStrings
using Printf
using LinearAlgebra

In [ ]:
default(xtickfontsize=14, ytickfontsize=14, ztickfontsize=14,
    guidefontsize=14, legendfontsize=12, lw=2, ms=8)

# Set Up Mesh

In [ ]:
@show n = 2^6; # number of points
x = LinRange(0, 2 * π, n + 1)[1:end-1];
m_vals =  [0:1:n÷2; -n÷2+1:1:-1];
dx = im *m_vals;

# Set Up Initial Condition

In [ ]:
u0 = @. exp(-(x-3).^2);
v0 = zeros(n);
plot(x, u0, label=L"u_0")
plot!(x, v0, label=L"v_0")
xlabel!(L"x")

# Set up Time Stepping

In [ ]:
Δt = 0.1;
n_steps = 100;

u = deepcopy(u0);
v = deepcopy(v0);
u_vals = [deepcopy(u)];
v_vals = [deepcopy(v)];
t_vals = [0.];

U = fft(u);
V = fft(v);
U_ = similar(U)
V_ = similar(V)

for i in 1:n_steps
    U_[2:end] = @. (m_vals[2:end] * U[2:end] - im * V[2:end]) / (2 * m_vals[2:end]) * exp(im * m_vals[2:end] * Δt) + (m_vals[2:end] * U[2:end] + im * V[2:end]) / (2 * m_vals[2:end]) * exp(-im * m_vals[2:end] * Δt)
    V_[2:end] = @. im * (m_vals[2:end] * U[2:end] - im * V[2:end]) / (2) * exp(im * m_vals[2:end] * Δt) - im * (m_vals[2:end] * U[2:end] + im * V[2:end]) / (2) * exp(-im * m_vals[2:end] * Δt)

    U_[1] = U[1] + V[1] * Δt;
    V_[1] = V[1];

    @. U = U_;
    @. V = V_;
   
    push!(u_vals, deepcopy(real(ifft(U))));
    push!(v_vals, deepcopy(real(ifft(V))))
    push!(t_vals, i * Δt);
end


In [ ]:
anim = @animate for (t, u) in zip(t_vals, u_vals)
    plot(x, u, label=L"u")
    title!(@sprintf("t = %.2f", t))
    xlabel!(L"$x$")
    ylims!(0, 1.5)
end

In [ ]:
gif(anim, fps=6)
